In [1]:
# DATA READ - REYNOLDS AND VELOCITY VALUES
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#import data

ANNDF = pd.read_excel('fxReynoldsV3.xlsx')

X = ANNDF.iloc[:,[1,3]].values
Y = ANNDF.iloc[:, 2].values.reshape(-1,1)

xtrain, xtest, ytrain,  ytest = train_test_split(X, Y, train_size = 0.75)

scalerx = StandardScaler()
scalery = StandardScaler()

Xtrain = scalerx.fit_transform(xtrain)
Ytrain = scalery.fit_transform(ytrain)



mlp = MLPRegressor(max_iter=1000)



In [2]:

# Define the KFold cross-validation
kf = KFold(n_splits=10, shuffle=True)

# Define the parameter grid
parameters = {
    'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (8, 8)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam']
}

# Define GridSearchCV
gsearchcv = GridSearchCV(estimator=mlp, param_grid=parameters, cv=kf)

# Fit the model
gsearchcv.fit(Xtrain, Ytrain)

# Get best score and parameters
bestac = gsearchcv.best_score_ * 100  # Convert to percentage
bestparam = gsearchcv.best_params_

print('Best Accuracy: {:.2f} %'.format(bestac))
print('Best Parameters:', bestparam)


c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Carlos\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1631: DataConversionWarning: A column-vecto

Best Accuracy: 98.63 %
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (8, 8), 'solver': 'adam'}


In [3]:
z = (scalerx.transform(xtest))
print(scalerx.inverse_transform(z))

[[5.01704100e+03 3.74500000e-01]
 [4.08598401e+03 3.74500000e-01]
 [4.77617791e+03 4.33100000e-01]
 ...
 [1.98920098e+03 3.74500000e-01]
 [3.75749174e+03 3.32600000e-01]
 [2.66384536e+03 4.16200000e-01]]


In [9]:
input_data = np.array([[1843, 0.4331]])

# Transformar os dados de entrada usando o scaler treinado
input_data_scaled = scalerx.transform(input_data)

# Fazer a previsão com o modelo treinado
predicted_scaled = gsearchcv.predict(input_data_scaled)

# Inverter a transformação do output (scaling)
predicted = scalery.inverse_transform(predicted_scaled.reshape(-1, 1))

Xtest_scaled = scalerx.transform(xtest)

ypred = gsearchcv.predict(Xtest_scaled).reshape(-1,1)

ypred_or = scalery.inverse_transform(ypred)

# Mostrar o resultado
print('Predicted value for [2000, 0.24]:', predicted[0][0])


Predicted value for [2000, 0.24]: 0.5097002814429804


In [13]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Definir os intervalos para Re e Stdp
Re_vals = np.linspace(1000, 8000, 100)  # Valores de Reynolds de 1000 a 8000
Stdp_vals = np.linspace(0.3326, 0.4331, 100)  # Valores de Stdp de 0.3326 a 0.4331

# Criar uma grade de Re e Stdp
Re_grid, Stdp_grid = np.meshgrid(Re_vals, Stdp_vals)
grid_points = np.c_[Re_grid.ravel(), Stdp_grid.ravel()]

# Prever os valores de 'f' usando a ANN nos pontos da grade
f_pred_scaled = gsearchcv.predict(scalerx.transform(grid_points))
f_pred = scalery.inverse_transform(f_pred_scaled.reshape(-1, 1))
f_pred_grid = f_pred.reshape(Re_grid.shape)

# Calcular a superfície usando a fórmula fornecida
f_formula = (4.3059 / np.log(Re_grid + 3.1918)) + ((0.30649 / Stdp_grid) - 0.80937)

# Criar o gráfico 3D para os resultados da ANN
surface_ann = go.Surface(x=Re_grid, y=Stdp_grid, z=f_pred_grid, colorscale='Viridis', name='ANN Surface')

# Criar o gráfico 3D para a fórmula fornecida, com cores em arco-íris
surface_formula = go.Surface(x=Re_grid, y=Stdp_grid, z=f_formula, colorscale='Rainbow', name='Formula Surface', opacity=0.7)

# Definir o layout do gráfico
layout = go.Layout(
    title='Superfície de Resultados da ANN e da Fórmula',
    scene=dict(
        xaxis=dict(title='Reynolds Number (Re)'),
        yaxis=dict(title='Stdp'),
        zaxis=dict(title='f')
    ),
    showlegend=True
)

# Criar a figura com ambas as superfícies
fig = go.Figure(data=[surface_ann, surface_formula], layout=layout)

# Mostrar o gráfico
pio.show(fig)


In [12]:

y = (4.3059 / np.log(1843 + 3.1918)) + ((0.30649 / 0.4331)-0.80937)
print(y)

0.4708217299026761


In [14]:
from sklearn.metrics import r2_score

# Achatar as superfícies em vetores 1D
f_ann_flat = f_pred_grid.ravel()
f_formula_flat = f_formula.ravel()

# Calcular o R^2
r2 = r2_score(f_ann_flat, f_formula_flat)

print('R² entre a superfície da ANN e a fórmula: {:.4f}'.format(r2))


R² entre a superfície da ANN e a fórmula: 0.9550
